## Como limpiar tus datos en Python

El artículo es creado como punto de partida para aprender a limpiar los datos de forma eficiente para así utilizarlo en proyectos personales. Además, se trabajará con el conjunto de datos de películas y programas de televisión en Netflix, el cual presenta ciertas inconsistencias y datos faltantes.

Primeramente importamos las librerías necesarias para trabajar en este conjunto de datos.

In [16]:
import pandas as pd
from datetime import datetime


Paso 1. Investigar los datos


Antes de realizar cualquier acción en el conjunto de datos, primero tenemos que revisar los datos para poder comprender con que variables se está trabajando, como se estructuran estos valores según la columna en la que se encuentran, para así tener una idea aproximada de lo que se necesita abordar o podrían ocasionar problemas en la fase de análisis. También se podrán eliminar ciertas columnas que no son necesarias, dependiendo del análisis que se quiere realizar.

1.1 Imprimir las primeras filas del conjunto de datos

Se imprimen las primeras 5 filas del conjunto de datos, se recomienda imprimir menos de 10 para no abrumarse al momento de analizar rápido el conjunto de datos.

In [17]:
# Importando el conjunto de datos
netflix_titles = pd.read_csv("netflix_titles.csv")
# Se imprimen las primeras 5 columnas del conjunto de datos
netflix_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Con esto se tendrá una buena idea de con que tipos de datos se estará tratando, que columnas requieren limpieza o transformaciones y que otros datos se pueden extraer.

1.2 Guardar las variables en una lista

In [18]:
# Trayendo las columnas del conjunto de datos
columns = list(netflix_titles.columns)
columns

['show_id',
 'type',
 'title',
 'director',
 'cast',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description']

In [19]:
# examining missing values
print("Missing values distribution: ")
print(netflix_titles.isnull().mean())
print("")

Missing values distribution: 
show_id         0.000000
type            0.000000
title           0.000000
director        0.299080
cast            0.093675
country         0.094357
date_added      0.001135
release_year    0.000000
rating          0.000454
duration        0.000341
listed_in       0.000000
description     0.000000
dtype: float64



In [20]:
# Verificando el tipo de dato de cada columna
print("Column datatypes: ")
print(netflix_titles.dtypes)

Column datatypes: 
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [21]:
# Trayendo todas las columinas con valores del tipo cadena/mezclas
str_cols = list(netflix_titles.columns)
str_cols.remove('release_year')

# Eliminación de los caracteres iniciales y finales de las columnas de tipo str
for i in str_cols:
    netflix_titles[i] = netflix_titles[i].str.strip()

In [22]:
# Nombre de las columnas
columns = ['director', 'cast', 'country', 'rating', 'date_added']

# Recorrer las columnas para rellenar las entradas con valores NaN con ""
for column in columns:
    netflix_titles[column] = netflix_titles[column].fillna("")

In [23]:
# Examinando filas con valores nulos de date_added column
rows = []
for i in range(len(netflix_titles)):
    if netflix_titles['date_added'].iloc[i] == "":
        rows.append(i)
    
# Examinar esas filas para confirmar el estado nulo
netflix_titles.loc[rows, :]
# Extracción de meses y años añadidos
month_added = []
year_added = []
for i in range(len(netflix_titles)):
    # Reemplazando valores NaN con 0
    if i in rows:
        month_added.append(0)
        year_added.append(0)
    else:
        date = netflix_titles['date_added'].iloc[i].split(" ")
        month_added.append(date[0])
        year_added.append(int(date[2]))
        
# Tornando el nombre de los meses en numeros de mes
for i, month in enumerate(month_added):
    if month != 0:
        datetime_obj = datetime.strptime(month, "%B")
        month_number = datetime_obj.month
        month_added[i] = month_number
        
# Checando todos los meses
print(set(month_added))
print(set(year_added))

# Insertando columnas del mes y ano en el conjunto de datos
netflix_titles.insert(7, "month_added", month_added, allow_duplicates = True)
netflix_titles.insert(8, "year_added", year_added, allow_duplicates = True)
netflix_titles.head()


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
{2016, 2017, 2018, 2019, 2020, 2021, 0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015}


,show_id,type,title,director,cast,country,date_added,month_added,year_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",9,2021,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",9,2021,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",,"September 24, 2021",9,2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,,,,"September 24, 2021",9,2021,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",9,2021,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [24]:
# separating original dataset to tv show and movie dataset respectively
shows = []
films = []

# looping through the dataset to identify rows that are TV shows and films
for i in range(len(netflix_titles)):
    if netflix_titles['type'].iloc[i] == "TV Show":
        shows.append(i)
    else:
        films.append(i)
 
# grouping rows that are TV shows
netflix_shows = netflix_titles.loc[shows, :]

#grouping rows that are films
netflix_films = netflix_titles.loc[films, :]

# reseting the index of the new datasets
netflix_shows = netflix_shows.set_index([pd.Index(range(0, len(netflix_shows)))])
netflix_films = netflix_films.set_index([pd.Index(range(0, len(netflix_films)))])


In [25]:
# get length of movie or number of seasons of show
def getDuration(data):
    count = 0
    durations = []
    for value in data:
	# filling in missing values
        if type(value) is float:
            durations.append(0)
        else:
            values = value.split(" ")
            durations.append(int(values[0]))
    return durations
    
# inserting new duration type column for shows (renamed column)
netflix_shows.insert(11, 'seasons', getDuration(netflix_shows['duration']))
netflix_shows = netflix_shows.drop(['duration'], axis = 1)
netflix_shows.head()

# inserting new duration type column for films (renamed column)
netflix_films.insert(11, 'length', getDuration(netflix_films['duration']))
netflix_films = netflix_films.drop(['duration'], axis = 1)
netflix_films.head()

,show_id,type,title,director,cast,country,date_added,month_added,year_added,release_year,rating,length,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",9,2021,2020,PG-13,90,Documentaries,"As her father nears the end of his life, filmm..."
1,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",,"September 24, 2021",9,2021,2021,PG,91,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
2,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",9,2021,1993,TV-MA,125,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
3,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",9,2021,2021,PG-13,104,"Comedies, Dramas",A woman adjusting to life after a loss contend...
4,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",9,2021,2021,TV-MA,127,"Dramas, International Movies",After most of her family is murdered in a terr...


In [26]:
# getting the unique ratings for films
netflix_films['rating'].unique()

array(['PG-13', 'PG', 'TV-MA', 'TV-PG', 'TV-14', 'TV-Y', 'R', 'TV-G',
       'TV-Y7', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', '',
       'TV-Y7-FV', 'UR'], dtype=object)

In [27]:
# getting the unique ratings for shows
netflix_shows['rating'].unique()

array(['TV-MA', 'TV-14', 'TV-Y7', 'TV-PG', 'TV-Y', 'TV-G', 'R', 'NR', '',
       'TV-Y7-FV'], dtype=object)

In [28]:
# printing more details of the rows that have incorrect ratings
incorrect_ratings = ['74 min', '84 min', '66 min']
for i in range(len(netflix_films)):
    if netflix_films['rating'].iloc[i] in incorrect_ratings:
        print(netflix_films.iloc[i])
        print("")

show_id                                                     s5542
type                                                        Movie
title                                             Louis C.K. 2017
director                                               Louis C.K.
cast                                                   Louis C.K.
country                                             United States
date_added                                          April 4, 2017
month_added                                                     4
year_added                                                   2017
release_year                                                 2017
rating                                                     74 min
length                                                          0
listed_in                                                  Movies
description     Louis C.K. muses on religion, eternal love, gi...
Name: 3562, dtype: object

show_id                                          

In [29]:
# getting the row indices
index = [3562, 3738, 3747]

# fixing the entries
for i in index:
    split_value = netflix_films['rating'].iloc[i].split(" ")
    length = split_value[0]
    netflix_films['length'].iloc[i] = length
    netflix_films['rating'].iloc[i] = "NR" 
    
    
# double checking the entries again
for i in index:
    print(netflix_films.iloc[i])

show_id                                                     s5542
type                                                        Movie
title                                             Louis C.K. 2017
director                                               Louis C.K.
cast                                                   Louis C.K.
country                                             United States
date_added                                          April 4, 2017
month_added                                                     4
year_added                                                   2017
release_year                                                 2017
rating                                                         NR
length                                                         74
listed_in                                                  Movies
description     Louis C.K. muses on religion, eternal love, gi...
Name: 3562, dtype: object
show_id                                           

C:\Users\ages0\AppData\Local\Temp\ipykernel_2332\1591530863.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_films['length'].iloc[i] = length
C:\Users\ages0\AppData\Local\Temp\ipykernel_2332\1591530863.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_films['rating'].iloc[i] = "NR"


In [30]:
# fixing the entries
for i in range(len(netflix_films)):
    if netflix_films['rating'].iloc[i] == "UR":
        netflix_films['rating'].iloc[i] = "NR"
        
# double checking
netflix_films['rating'].unique()


C:\Users\ages0\AppData\Local\Temp\ipykernel_2332\3877169165.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_films['rating'].iloc[i] = "NR"


array(['PG-13', 'PG', 'TV-MA', 'TV-PG', 'TV-14', 'TV-Y', 'R', 'TV-G',
       'TV-Y7', 'G', 'NC-17', 'NR', '', 'TV-Y7-FV'], dtype=object)

In [31]:

# function to get unique values of a column
def getUnique(data):
    unique_values = set()
    for value in data:
        if type(value) is float:
            unique_values.add(None)
        else:
            values = value.split(", ")
            for i in values:
                unique_values.add(i)
    return list(unique_values)

In [32]:
# getting unique country names
unique_countries = getUnique(netflix_titles['country'])
unique_countries

['',
 'Netherlands',
 'Azerbaijan',
 'Mongolia',
 'Uganda',
 'Singapore',
 'Croatia',
 'Samoa',
 'Colombia',
 'United States,',
 'Latvia',
 'Botswana',
 'Luxembourg',
 'Iran',
 'Venezuela',
 'Mauritius',
 'Albania',
 'Liechtenstein',
 'Sudan',
 'Belarus',
 'Portugal',
 'Ireland',
 'West Germany',
 'Chile',
 'Hong Kong',
 'Pakistan',
 'Algeria',
 'Cameroon',
 'Nepal',
 'Egypt',
 'Senegal',
 'Cuba',
 'United Kingdom',
 'Peru',
 'Poland',
 'Montenegro',
 'Mexico',
 'Jamaica',
 'Sweden',
 'Burkina Faso',
 'Iraq',
 'Germany',
 'Romania',
 'Malawi',
 'Sri Lanka',
 'Vatican City',
 'Bangladesh',
 'Cambodia,',
 'Iceland',
 'Canada',
 'Mozambique',
 'Morocco',
 'China',
 'Palestine',
 'Cyprus',
 'Kazakhstan',
 'Bulgaria',
 'Switzerland',
 'Georgia',
 'Hungary',
 'Ecuador',
 'Denmark',
 'Bahamas',
 'Serbia',
 'Nigeria',
 'Israel',
 'Argentina',
 'France',
 'Poland,',
 'Czech Republic',
 'South Korea',
 'Zimbabwe',
 'Slovenia',
 'Malaysia',
 'Italy',
 'Guatemala',
 'Paraguay',
 'United Kingdom,',

In [33]:
# converting soviet union to russia and east/west germany to germany
for i in range(len(netflix_titles)):
    if type(netflix_titles['country'].iloc[i]) is not float:
        countries = netflix_titles['country'].iloc[i].split(", ")
        for j in range(len(countries)):
            if "Germany" in countries[j]:
                countries[j] = "Germany"
            elif "Soviet Union" in countries[j]:
                countries[j] = "Russia"
        netflix_titles['country'].iloc[i] = ", ".join(countries)

C:\Users\ages0\AppData\Local\Temp\ipykernel_2332\1037538766.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_titles['country'].iloc[i] = ", ".join(countries)


In [34]:
# getting unique film genres
unique_genres_films = getUnique(netflix_films['listed_in'])
unique_genres_films

['Music & Musicals',
 'Horror Movies',
 'Thrillers',
 'Comedies',
 'Children & Family Movies',
 'International Movies',
 'Documentaries',
 'Movies',
 'Anime Features',
 'Independent Movies',
 'Romantic Movies',
 'LGBTQ Movies',
 'Sci-Fi & Fantasy',
 'Action & Adventure',
 'Sports Movies',
 'Classic Movies',
 'Dramas',
 'Stand-Up Comedy',
 'Faith & Spirituality',
 'Cult Movies']

In [35]:
# getting unique show genres
unique_genres_shows = getUnique(netflix_shows['listed_in'])
unique_genres_shows

['International TV Shows',
 'TV Horror',
 'Stand-Up Comedy & Talk Shows',
 'TV Action & Adventure',
 'Docuseries',
 'Teen TV Shows',
 'TV Mysteries',
 'TV Sci-Fi & Fantasy',
 'Korean TV Shows',
 'Classic & Cult TV',
 'Science & Nature TV',
 'TV Dramas',
 'Crime TV Shows',
 'Anime Series',
 'TV Comedies',
 'Romantic TV Shows',
 'TV Thrillers',
 'TV Shows',
 'British TV Shows',
 'Spanish-Language TV Shows',
 "Kids' TV",
 'Reality TV']

In [36]:
# checking for TV shows
# replace netflix_shows with netflix_films to check for movies
count = 0
index = []
for i, value in enumerate(netflix_shows['listed_in']):
    genres = value.split(", ")
    if "TV Shows" in genres:
        count += 1
        index.append(i)
print("count %s" %count)
print("index %s" %index)

count 16
index [59, 110, 272, 286, 452, 599, 991, 1432, 1548, 1808, 1840, 2107, 2160, 2190, 2465, 2559]


In [37]:
# printing the first 5 rows of all rows that have TV Shows as its genre
netflix_shows.iloc[index[0:5]]


,show_id,type,title,director,cast,country,date_added,month_added,year_added,release_year,rating,seasons,listed_in,description
59,s149,TV Show,HQ Barbers,Gerhard Mostert,"Hakeem Kae-Kazim, Chioma Omeruah, Orukotan Ade...",,"September 1, 2021",9,2021,2020,TV-14,1,TV Shows,When a family run barber shop in the heart of ...
110,s298,TV Show,Navarasa,"Bejoy Nambiar, Priyadarshan, Karthik Narain, V...","Suriya, Vijay Sethupathi, Revathy, Prakash Raj...",India,"August 6, 2021",8,2021,2021,TV-MA,1,TV Shows,"From amusement to awe, the nine human emotions..."
272,s727,TV Show,Metallica: Some Kind of Monster,"Joe Berlinger, Bruce Sinofsky","James Hetfield, Lars Ulrich, Kirk Hammett, Rob...",United States,"June 13, 2021",6,2021,2014,TV-MA,1,TV Shows,This collection includes the acclaimed rock do...
286,s772,TV Show,Pretty Guardian Sailor Moon Eternal The Movie,Chiaki Kon,"Kotono Mitsuishi, Hisako Kanemoto, Rina Satou,...",,"June 3, 2021",6,2021,2021,TV-14,1,TV Shows,When a dark power enshrouds the Earth after a ...
452,s1332,TV Show,Five Came Back: The Reference Films,,,United States,"February 9, 2021",2,2021,1945,TV-MA,1,TV Shows,This collection includes 12 World War II-era p...


In [38]:
# printing the first 5 rows of all rows that have Movies as its genre
netflix_films.iloc[index[0:5]]

,show_id,type,title,director,cast,country,date_added,month_added,year_added,release_year,rating,length,listed_in,description
59,s108,Movie,A Champion Heart,David de Vos,"Mandy Grace, David de Vos, Donna Rusch, Devan ...",United States,"September 4, 2021",9,2021,2018,G,90,"Children & Family Movies, Dramas",When a grieving teen must work off her debt to...
110,s175,Movie,Tears of the Sun,Antoine Fuqua,"Bruce Willis, Monica Bellucci, Cole Hauser, Ea...",United States,"September 1, 2021",9,2021,2003,R,121,"Action & Adventure, Dramas",A Navy SEAL is sent to a war-torn African jung...
272,s420,Movie,Chhota Bheem: Bheem vs Aliens,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",,"July 22, 2021",7,2021,2010,TV-Y7,69,"Children & Family Movies, Sports Movies",Space invaders have kidnapped Dholakpur’s king...
286,s439,Movie,2 Weeks in Lagos,Kathryn Fasegha,"Beverly Naya, Mawuli Gavor, Ajoke Silva, Jide ...",,"July 16, 2021",7,2021,2020,TV-PG,107,"Dramas, International Movies, Romantic Movies",A businessman returns home to Nigeria and fall...
452,s723,Movie,Sir! No Sir!,David Zeiger,Troy Garity,United States,"June 15, 2021",6,2021,2005,TV-MA,84,Documentaries,This documentary chronicles the largely forgot...


Todo lo explicado en este notebook pertenece al articulo de Huong Ngo
https://towardsdatascience.com/how-to-clean-your-data-in-python-8f178638b98d